In [64]:
import numpy as np
import polars as pl
from ebnerd_benchmark_lib.utils._constants import DEFAULT_USER_COL

import os
from pathlib import Path

# read in data

train_val_base = Path(os.getcwd()).parent / "data-merged" / "merged" / "1-ebnerd_demo_(20MB)"
# train_val_base = Path(os.getcwd()).parent / "data-merged" / "merged" / "3-ebnerd_large_(3.0GB)"
test_base = Path(os.getcwd()).parent / "data-merged" / "merged" / "5-ebnerd_testset_(1.5GB)"
assert train_val_base.exists() and test_base.exists()

train_behaviors: pl.LazyFrame = pl.scan_parquet(train_val_base / "train" / "behaviors.parquet")
train_history: pl.LazyFrame = pl.scan_parquet(train_val_base / "train" / "history.parquet")
train_user: pl.LazyFrame = train_behaviors.join(train_history, on=DEFAULT_USER_COL, how="inner")
train_articles: pl.LazyFrame = pl.scan_parquet(train_val_base / "articles.parquet")

val_behaviors: pl.LazyFrame = pl.scan_parquet(train_val_base / "validation" / "behaviors.parquet")
val_history: pl.LazyFrame = pl.scan_parquet(train_val_base / "validation" / "history.parquet")
val_user: pl.LazyFrame = val_behaviors.join(val_history, on=DEFAULT_USER_COL, how="inner")
val_articles: pl.LazyFrame = pl.scan_parquet(train_val_base / "articles.parquet")

test_behaviors: pl.LazyFrame = pl.scan_parquet(test_base / "test" / "behaviors.parquet")
test_history: pl.LazyFrame = pl.scan_parquet(test_base / "test" / "history.parquet")
test_user: pl.LazyFrame = test_behaviors.join(test_history, on=DEFAULT_USER_COL, how="inner")
test_articles: pl.LazyFrame = pl.scan_parquet(test_base / "articles.parquet")

# check if intersection as expected

common_cols = set(train_behaviors.columns).intersection(set(train_history.columns))
assert list(common_cols)[0] == DEFAULT_USER_COL

train_user.head(5).collect()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32,list[datetime[μs]],list[f32],list[i32],list[f32]
48401,null,2023-05-21 21:06:50,21.0,null,2,"[9774516, 9771051, … 9759966]",[9759966],22779,false,null,null,null,false,21,16.0,27.0,"[2023-04-27 09:05:54, 2023-04-27 09:06:09, … 2023-05-18 06:26:39]","[28.0, 17.0, … 15.0]","[9738452, 9737521, … 9770541]","[5.0, 4.0, … 7.0]"
152513,9778745,2023-05-24 07:31:26,30.0,100.0,1,"[9778669, 9778736, … 9777397]",[9778661],150224,false,null,null,null,false,298,2.0,48.0,"[2023-04-29 11:34:06, 2023-04-29 11:34:25, … 2023-05-18 06:13:47]","[100.0, 49.0, … 24.0]","[9740087, 9741986, … 9735909]","[18.0, 244.0, … 7.0]"
155390,null,2023-05-24 07:30:33,45.0,null,1,"[9778369, 9777856, … 9778448]",[9777856],160892,false,null,null,null,false,401,215.0,100.0,"[2023-04-27 09:10:33, 2023-04-27 09:20:25, … 2023-05-17 15:51:19]","[100.0, 20.0, … 100.0]","[9738557, 9738211, … 9770178]","[583.0, 257.0, … 158.0]"
214679,null,2023-05-23 05:25:40,33.0,null,2,"[9776715, 9776406, … 9776855]",[9776566],1001055,false,null,null,null,false,1357,40.0,47.0,"[2023-04-27 12:12:45, 2023-04-27 12:13:30, … 2023-05-18 05:31:44]","[100.0, 100.0, … 28.0]","[9738777, 9738663, … 9769981]","[35.0, 62.0, … 16.0]"
214681,null,2023-05-23 05:31:54,21.0,null,2,"[9775202, 9776855, … 9776570]",[9776553],1001055,false,null,null,null,false,1358,5.0,49.0,"[2023-04-27 12:12:45, 2023-04-27 12:13:30, … 2023-05-18 05:31:44]","[100.0, 100.0, … 28.0]","[9738777, 9738663, … 9769981]","[35.0, 62.0, … 16.0]"


In [59]:
# df = pl.concat(
#     (rows.join(train_behaviors.collect(), on=DEFAULT_USER_COL, how="left")
#      for rows in train_behaviors.collect().iter_slices())
# )
# df.head(5)


# cols in common
common_cols = set(train_behaviors.columns).intersection(set(train_history.columns))
print(common_cols, DEFAULT_USER_COL)

# merge
train = train_behaviors.join(train_history, on=DEFAULT_USER_COL, how="inner").collect()
train.head(5)

{'user_id'} user_id


impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32,list[datetime[μs]],list[f32],list[i32],list[f32]
48401,null,2023-05-21 21:06:50,21.0,null,2,"[9774516, 9771051, … 9759966]",[9759966],22779,false,null,null,null,false,21,16.0,27.0,"[2023-04-27 09:05:54, 2023-04-27 09:06:09, … 2023-05-18 06:26:39]","[28.0, 17.0, … 15.0]","[9738452, 9737521, … 9770541]","[5.0, 4.0, … 7.0]"
152513,9778745,2023-05-24 07:31:26,30.0,100.0,1,"[9778669, 9778736, … 9777397]",[9778661],150224,false,null,null,null,false,298,2.0,48.0,"[2023-04-29 11:34:06, 2023-04-29 11:34:25, … 2023-05-18 06:13:47]","[100.0, 49.0, … 24.0]","[9740087, 9741986, … 9735909]","[18.0, 244.0, … 7.0]"
155390,null,2023-05-24 07:30:33,45.0,null,1,"[9778369, 9777856, … 9778448]",[9777856],160892,false,null,null,null,false,401,215.0,100.0,"[2023-04-27 09:10:33, 2023-04-27 09:20:25, … 2023-05-17 15:51:19]","[100.0, 20.0, … 100.0]","[9738557, 9738211, … 9770178]","[583.0, 257.0, … 158.0]"
214679,null,2023-05-23 05:25:40,33.0,null,2,"[9776715, 9776406, … 9776855]",[9776566],1001055,false,null,null,null,false,1357,40.0,47.0,"[2023-04-27 12:12:45, 2023-04-27 12:13:30, … 2023-05-18 05:31:44]","[100.0, 100.0, … 28.0]","[9738777, 9738663, … 9769981]","[35.0, 62.0, … 16.0]"
214681,null,2023-05-23 05:31:54,21.0,null,2,"[9775202, 9776855, … 9776570]",[9776553],1001055,false,null,null,null,false,1358,5.0,49.0,"[2023-04-27 12:12:45, 2023-04-27 12:13:30, … 2023-05-18 05:31:44]","[100.0, 100.0, … 28.0]","[9738777, 9738663, … 9769981]","[35.0, 62.0, … 16.0]"
